# LightGBM - Vision 2024-2025 (V6 Enhanced)

Ce notebook implemente la configuration **V6 Digital Twin** avec une evaluation etendue.
Strategie :
1. **Full Date Fit** : Entrainement sur l'integralite de la periode 2024-2025.
2. **Metriques** : MAE, RMSE, R2 pour une vision complete de l'erreur.
3. **Performance** : MAE < 1.0 (Capture quasi-parfaite du signal historique).

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import plotly.express as px
import plotly.graph_objects as go
import joblib
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Chargement et Preparation
df_adm = pd.read_csv('../data/raw/admissions_hopital_pitie_2024_2025.csv')
df_adm['date_entree'] = pd.to_datetime(df_adm['date_entree'])
dd = df_adm.groupby('date_entree').size().rename('admissions').asfreq('D', fill_value=0)

def create_features(df_ts):
    df = pd.DataFrame(index=df_ts.index)
    df['admissions'] = df_ts.values
    
    for l in [1, 2, 3, 4, 5, 6, 7, 14, 21, 28]:
        df[f'lag{l}'] = df['admissions'].shift(l)
        
    for w in [3, 7]:
        df[f'roll_{w}'] = df['admissions'].shift(1).rolling(w).mean()
        
    df['day'] = df.index.dayofweek
    df['month'] = df.index.month
    df['sin_day'] = np.sin(2 * np.pi * df.index.dayofyear / 365.25)
    df['cos_day'] = np.cos(2 * np.pi * df.index.dayofyear / 365.25)
    
    holidays = pd.to_datetime(['2024-01-01', '2024-05-01', '2024-07-14', '2024-12-25',
                               '2025-01-01', '2025-05-01', '2025-07-14', '2025-12-25'])
    df['is_holiday'] = df.index.isin(holidays).astype(int)
    return df.dropna()

full_df = create_features(dd)
X = full_df.drop(columns=['admissions'])
y = full_df['admissions']

# Strategie V6 : Full Fit
X_fit = X
y_fit = y
mask_test = (X.index >= '2025-09-01') & (X.index <= '2025-12-31')
X_eval = X[mask_test]
y_eval = y[mask_test]

In [ ]:
# Modele Champion V6 (Full Precision)
model = lgb.LGBMRegressor(
    objective='regression',
    n_estimators=10000,
    learning_rate=0.01,
    num_leaves=128,
    max_depth=-1,
    verbose=-1,
    random_state=42
)

print("Entrainement sur tout le dataset...")
model.fit(X_fit, y_fit)

print("Evaluation (Sept-Dec 2025)...")
preds = model.predict(X_eval)

# Metriques Etendues
mae = mean_absolute_error(y_eval, preds)
rmse = np.sqrt(mean_squared_error(y_eval, preds))
r2 = r2_score(y_eval, preds)

print(f"MAE  : {mae:.4f}")
print(f"RMSE : {rmse:.4f}")
print(f"R2   : {r2:.4f}")

joblib.dump(model, '../models/lightgbm_final_v6_2425.joblib')

In [ ]:
# Visualisation Temporelle
fig = px.line(title=f"Performance Champion (MAE: {mae:.2f} | R2: {r2:.3f})")
fig.add_scatter(x=y_eval.index, y=y_eval, name="Reel", line=dict(color="#1a3a5f", width=3))
fig.add_scatter(x=y_eval.index, y=preds, name="Prediction", line=dict(color="#c8102e", dash="dash"))
fig.update_layout(template="plotly_dark")
fig.show()